In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from utils import *

## Solution to exercises in Deep Learning chapter

1. Build a DNN with five hidden layers of 100 neurons each, He initialization, and the ELU activation function.

In [2]:
he_init = tf.variance_scaling_initializer()

In [3]:
def dnn(inputs, n_hidden_layers=5, n_neurons=100, name=None, activation=tf.nn.elu, initializer=he_init):
    with tf.variable_scope(name, 'dnn'):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
                                    kernel_initializer=initializer,
                                    name='hidden%d' % (layer + 1))
        return inputs

In [4]:
n_inputs = 28 * 28 # MNIST
n_outputs = 5

reset_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')

dnn_outputs = dnn(X)

logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name='logits')
Y_proba = tf.nn.softmax(logits, name='Y_proba')

2. Using Adam optimization and early stopping, try training it on MNIST but only on digits 0 to 4, as we will use transfer learning for digits 5 to 9 in the next exercise. You will need a softmax output layer with five neurons, and as always make sure to save checkpoints at regular intervals and save the final model so you can reuse it later.

Let's complete the graph with the cost function, the training op, and all the other usual components:

In [6]:
learning_rate = 0.01

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name='loss')

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss, name='training_op')

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')

init = tf.global_variables_initializer()
saver = tf.train.Saver()

Now let's create the training set, validation and test set (we need the validation set to implement early stopping):

In [8]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [9]:
X_train1 = X_train[y_train < 5]
y_train1 = y_train[y_train < 5]
X_valid1 = X_valid[y_valid < 5]
y_valid1 = y_valid[y_valid < 5]
X_test1 = X_test[y_test < 5]
y_test1 = y_test[y_test < 5]

In [10]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train1))
        for rnd_indices in np.array_split(rnd_idx, len(X_train1) // batch_size):
            X_batch, y_batch = X_train1[rnd_indices], y_train1[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X:X_valid1, y:y_valid1})
        if loss_val < best_loss:
            save_path = saver.save(sess, '/tmp/my_mnist_model_0_to_4.ckpt')
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print('Early stopping!')
                break
        print('{}\t validation loss: {:.6f}\t Best loss: {:.6f}\t Accuracy: {:.2f}%'.format(
            epoch,loss_val, best_loss, acc_val * 100))
        
with tf.Session() as sess:
    saver.restore(sess, '/tmp/my_mnist_model_0_to_4.ckpt')
    acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
    print('Final test accuracy: {:.2f}%'.format(acc_test * 100))

0	 validation loss: 0.112589	 Best loss: 0.112589	 Accuracy: 96.87%
1	 validation loss: 0.283811	 Best loss: 0.112589	 Accuracy: 97.07%
2	 validation loss: 0.107586	 Best loss: 0.107586	 Accuracy: 97.22%
3	 validation loss: 0.119034	 Best loss: 0.107586	 Accuracy: 97.69%
4	 validation loss: 0.103963	 Best loss: 0.103963	 Accuracy: 98.05%
5	 validation loss: 0.307087	 Best loss: 0.103963	 Accuracy: 94.88%
6	 validation loss: 0.199657	 Best loss: 0.103963	 Accuracy: 96.76%
7	 validation loss: 0.120272	 Best loss: 0.103963	 Accuracy: 97.89%
8	 validation loss: 0.107294	 Best loss: 0.103963	 Accuracy: 97.93%
9	 validation loss: 0.786723	 Best loss: 0.103963	 Accuracy: 84.95%
10	 validation loss: 0.169237	 Best loss: 0.103963	 Accuracy: 96.76%
11	 validation loss: 0.194776	 Best loss: 0.103963	 Accuracy: 95.82%
12	 validation loss: 0.184340	 Best loss: 0.103963	 Accuracy: 97.30%
13	 validation loss: 0.358477	 Best loss: 0.103963	 Accuracy: 96.91%
14	 validation loss: 0.504995	 Best loss: 0.

3. Tune the hyperparameters using cross-validation and see what precision you can achieve.

Let's create a DNNClassifier class, compatible with Scikit-Learn's RandomizedSearchCV class, to perform hyperparameter tuning. Here are the key points of this implementation:

* the `__init__()` method (constructor) does nothing more than create instance variables for each of the hyperparameters.
* the `fit()` method creates the graph, starts a session and trains the model:
    * it calls the `_build_graph()` method to build the graph (much lile the graph we defined earlier). Once this method is done creating the graph, it saves all the important operations as instance variables for easy access by other methods.
    * the `_dnn()` method builds the hidden layers, just like the `dnn()` function above, but also with support for batch normalization and dropout (for the next exercises).
    * if the `fit()` method is given a validation set (`X_valid` and `y_valid`), then it implements early stopping. This implementation does not save the best model to disk, but rather to memory: it uses the `_get_model_params()` method to get all the graph's variables and their values, and the `_restore_model_params()` method to restore the variable values (of the best model found). This trick helps speed up training.
    * After the `fit()` method has finished training the model, it keeps the session open so that predictions can be made quickly, without having to save a model to disk and restore it for every prediction. You can close the session by calling the `close_session()` method.
* the `predict_proba()` method uses the trained model to predict the class probabilities.
* the `predict()` method calls `predict_proba()` and returns the class with the highest probability, for each instance.

In [17]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class = tf.train.AdamOptimizer,
                learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=he_init,
                batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters.
        """
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        self.initializer = initializer
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None
        
    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout.
        """
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons, kernel_initializer=self.initializer,
                                    name='hidden%d' % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                      training=self._training)
            inputs = self.activation(inputs, name='hidden%d_out' % (layer + 1))
        return inputs
    
    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)
            
        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
        y = tf.placeholder(tf.int32, shape=(None), name='y')
        
        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None
            
        dnn_outputs = self._dnn(X)
        
        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name='logits')
        Y_proba = tf.nn.softmax(logits, name='Y_proba')
        
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        loss = tf.reduce_mean(xentropy, name='loss')
        
        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)
        
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')
        
        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
        
        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver
    
    def close_session(self):
        if self._session:
            self._session.close()
            
    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}
    
    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + '/Assign')
                     for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)
        
    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()
        
        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label] for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            
        # needed in case of early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                feed_dict={self._X: X_valid,
                                                           self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print('{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%'.format(
                    epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print('Early stopping!')
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                    feed_dict={self._X: X_batch,
                                                               self._y: y_batch})
                    print('{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%'.format(
                    epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self
        
    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError('This %s instance is not fitted yet' % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X:X})
        
    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)
    
    def save(self, path):
        self._saver.save(self._session, path)

In [18]:
dnn_clf = DNNClassifier(random_state=42)
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

0	Validation loss: 0.112589	Best loss: 0.112589	Accuracy: 96.87%
1	Validation loss: 0.283811	Best loss: 0.112589	Accuracy: 97.07%
2	Validation loss: 0.107586	Best loss: 0.107586	Accuracy: 97.22%
3	Validation loss: 0.119034	Best loss: 0.107586	Accuracy: 97.69%
4	Validation loss: 0.103963	Best loss: 0.103963	Accuracy: 98.05%
5	Validation loss: 0.307087	Best loss: 0.103963	Accuracy: 94.88%
6	Validation loss: 0.199657	Best loss: 0.103963	Accuracy: 96.76%
7	Validation loss: 0.120272	Best loss: 0.103963	Accuracy: 97.89%
8	Validation loss: 0.107294	Best loss: 0.103963	Accuracy: 97.93%
9	Validation loss: 0.786723	Best loss: 0.103963	Accuracy: 84.95%
10	Validation loss: 0.169237	Best loss: 0.103963	Accuracy: 96.76%
11	Validation loss: 0.194776	Best loss: 0.103963	Accuracy: 95.82%
12	Validation loss: 0.184340	Best loss: 0.103963	Accuracy: 97.30%
13	Validation loss: 0.358477	Best loss: 0.103963	Accuracy: 96.91%
14	Validation loss: 0.504995	Best loss: 0.103963	Accuracy: 78.89%
15	Validation loss: 

DNNClassifier(activation=<function elu at 0x7faad437d400>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7faaf003ec50>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [19]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9857949017318545

In [20]:
from sklearn.model_selection import RandomizedSearchCV

def leaky_relu(alpha=0.01):
    def parameterized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parameterized_leaky_relu

param_distribs = {
    'n_neurons': [10, 30, 50, 70, 90, 100, 120, 140, 160],
    'batch_size': [10, 50, 100, 500],
    'learning_rate': [0.01, 0.02, 0.05, 0.1],
    'activation': [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
}

rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
                               cv=3, random_state=42, verbose=2)
rnd_search.fit(X_train1, y_train1, X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_neurons=10, learning_rate=0.05, batch_size=100, activation=<function elu at 0x7faad437d400> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0	Validation loss: 0.205651	Best loss: 0.205651	Accuracy: 94.49%
1	Validation loss: 0.130858	Best loss: 0.130858	Accuracy: 96.01%
2	Validation loss: 0.113077	Best loss: 0.113077	Accuracy: 96.68%
3	Validation loss: 0.095059	Best loss: 0.095059	Accuracy: 97.65%
4	Validation loss: 0.106685	Best loss: 0.095059	Accuracy: 97.34%
5	Validation loss: 3.013369	Best loss: 0.095059	Accuracy: 74.39%
6	Validation loss: 1.636230	Best loss: 0.095059	Accuracy: 19.08%
7	Validation loss: 1.629931	Best loss: 0.095059	Accuracy: 22.01%
8	Validation loss: 1.626461	Best loss: 0.095059	Accuracy: 19.27%
9	Validation loss: 1.619111	Best loss: 0.095059	Accuracy: 19.27%
10	Validation loss: 1.611083	Best loss: 0.095059	Accuracy: 22.01%
11	Validation loss: 1.611269	Best loss: 0.095059	Accuracy: 20.91%
12	Validation loss: 1.636175	Best loss: 0.095059	Accuracy: 20.91%
13	Validation loss: 1.637586	Best loss: 0.095059	Accuracy: 18.73%
14	Validation loss: 1.620345	Best loss: 0.095059	Accuracy: 22.01%
15	Validation loss: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.4s remaining:    0.0s


0	Validation loss: 0.133521	Best loss: 0.133521	Accuracy: 96.36%
1	Validation loss: 0.158360	Best loss: 0.133521	Accuracy: 96.64%
2	Validation loss: 0.213154	Best loss: 0.133521	Accuracy: 95.82%
3	Validation loss: 0.164181	Best loss: 0.133521	Accuracy: 96.72%
4	Validation loss: 0.116952	Best loss: 0.116952	Accuracy: 97.42%
5	Validation loss: 0.100043	Best loss: 0.100043	Accuracy: 97.62%
6	Validation loss: 0.108463	Best loss: 0.100043	Accuracy: 97.46%
7	Validation loss: 0.178521	Best loss: 0.100043	Accuracy: 96.72%
8	Validation loss: 1.320603	Best loss: 0.100043	Accuracy: 36.63%
9	Validation loss: 1.298051	Best loss: 0.100043	Accuracy: 37.22%
10	Validation loss: 1.290452	Best loss: 0.100043	Accuracy: 36.08%
11	Validation loss: 1.246598	Best loss: 0.100043	Accuracy: 38.94%
12	Validation loss: 1.190964	Best loss: 0.100043	Accuracy: 38.94%
13	Validation loss: 1.400566	Best loss: 0.100043	Accuracy: 31.00%
14	Validation loss: 1.356373	Best loss: 0.100043	Accuracy: 34.79%
15	Validation loss: 

...

28	Validation loss: 0.269974	Best loss: 0.269974	Accuracy: 97.11%
29	Validation loss: 0.334603	Best loss: 0.269974	Accuracy: 96.76%
30	Validation loss: 0.362922	Best loss: 0.269974	Accuracy: 96.52%
31	Validation loss: 0.333291	Best loss: 0.269974	Accuracy: 96.76%
32	Validation loss: 0.293844	Best loss: 0.269974	Accuracy: 97.34%
33	Validation loss: 0.296734	Best loss: 0.269974	Accuracy: 97.22%
34	Validation loss: 0.273486	Best loss: 0.269974	Accuracy: 96.95%
35	Validation loss: 0.283241	Best loss: 0.269974	Accuracy: 96.95%
36	Validation loss: 0.464781	Best loss: 0.269974	Accuracy: 96.17%
37	Validation loss: 0.269623	Best loss: 0.269623	Accuracy: 96.95%
38	Validation loss: 0.240988	Best loss: 0.240988	Accuracy: 97.07%
39	Validation loss: 0.441261	Best loss: 0.240988	Accuracy: 96.68%
40	Validation loss: 0.343695	Best loss: 0.240988	Accuracy: 96.99%
41	Validation loss: 0.264561	Best loss: 0.240988	Accuracy: 96.87%
42	Validation loss: 0.310169	Best loss: 0.240988	Accuracy: 96.79%
43	Validat

86	Validation loss: 89227.085938	Best loss: 1.282186	Accuracy: 93.28%
Early stopping!
[CV]  n_neurons=90, learning_rate=0.1, batch_size=500, activation=<function leaky_relu.<locals>.parameterized_leaky_relu at 0x7faa38417d08>, total=  30.1s


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 217.8min finished


0	Validation loss: 0.069587	Best loss: 0.069587	Accuracy: 98.12%
1	Validation loss: 0.046135	Best loss: 0.046135	Accuracy: 98.44%
2	Validation loss: 0.047429	Best loss: 0.046135	Accuracy: 98.40%
3	Validation loss: 0.033808	Best loss: 0.033808	Accuracy: 98.79%
4	Validation loss: 0.048578	Best loss: 0.033808	Accuracy: 98.44%
5	Validation loss: 0.033306	Best loss: 0.033306	Accuracy: 98.94%
6	Validation loss: 0.048725	Best loss: 0.033306	Accuracy: 98.59%
7	Validation loss: 0.044024	Best loss: 0.033306	Accuracy: 98.79%
8	Validation loss: 0.045653	Best loss: 0.033306	Accuracy: 98.94%
9	Validation loss: 0.043885	Best loss: 0.033306	Accuracy: 98.83%
10	Validation loss: 0.036913	Best loss: 0.033306	Accuracy: 98.79%
11	Validation loss: 0.045165	Best loss: 0.033306	Accuracy: 98.75%
12	Validation loss: 0.056894	Best loss: 0.033306	Accuracy: 98.87%
13	Validation loss: 0.041044	Best loss: 0.033306	Accuracy: 98.94%
14	Validation loss: 0.062070	Best loss: 0.033306	Accuracy: 98.48%
15	Validation loss: 

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=DNNClassifier(activation=<function elu at 0x7faad437d400>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7faaf003ec50>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42),
          fit_params=None, iid='warn', n_iter=50, n_jobs=None,
          param_distributions={'n_neurons': [10, 30, 50, 70, 90, 100, 120, 140, 160], 'batch_size': [10, 50, 100, 500], 'learning_rate': [0.01, 0.02, 0.05, 0.1], 'activation': [<function relu at 0x7faad4328730>, <function elu at 0x7faad437d400>, <function leaky_relu.<locals>.parameterized_leaky_relu at 0x7faa38417d90>, <function leaky_relu.<locals>.parameterized_leaky_relu at 0x7faa38417d08>]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          r

In [21]:
rnd_search.best_params_

{'n_neurons': 90,
 'learning_rate': 0.01,
 'batch_size': 500,
 'activation': <function __main__.leaky_relu.<locals>.parameterized_leaky_relu(z, name=None)>}

In [22]:
y_pred = rnd_search.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9887137575403775

In [23]:
rnd_search.best_estimator_.save('/tmp/my_best_mnist_model_0_to_4')

4. Now try adding Batch normalization and compare the learning curves: is it converging faster than before? Does it produce a better model?

Let's train the best model found, once again, to see how fast it converges

In [24]:
dnn_clf = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=500, learning_rate=0.01,
                       n_neurons=140, random_state=42)
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

0	Validation loss: 0.083541	Best loss: 0.083541	Accuracy: 97.54%
1	Validation loss: 0.053076	Best loss: 0.053076	Accuracy: 98.32%
2	Validation loss: 0.040197	Best loss: 0.040197	Accuracy: 98.59%
3	Validation loss: 0.067104	Best loss: 0.040197	Accuracy: 97.97%
4	Validation loss: 0.048440	Best loss: 0.040197	Accuracy: 98.48%
5	Validation loss: 0.048565	Best loss: 0.040197	Accuracy: 98.55%
6	Validation loss: 0.042532	Best loss: 0.040197	Accuracy: 99.06%
7	Validation loss: 0.040291	Best loss: 0.040197	Accuracy: 99.10%
8	Validation loss: 0.058002	Best loss: 0.040197	Accuracy: 98.48%
9	Validation loss: 0.043863	Best loss: 0.040197	Accuracy: 98.87%
10	Validation loss: 0.060558	Best loss: 0.040197	Accuracy: 98.75%
11	Validation loss: 0.075632	Best loss: 0.040197	Accuracy: 98.91%
12	Validation loss: 0.053281	Best loss: 0.040197	Accuracy: 98.75%
13	Validation loss: 0.038663	Best loss: 0.038663	Accuracy: 99.06%
14	Validation loss: 0.041070	Best loss: 0.038663	Accuracy: 99.14%
15	Validation loss: 

DNNClassifier(activation=<function leaky_relu.<locals>.parameterized_leaky_relu at 0x7faa383c3ea0>,
       batch_norm_momentum=None, batch_size=500, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7faaf003ec50>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=140,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [25]:
y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9922163845106052

Now with batch normalization:

In [26]:
dnn_clf_bn = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=500, learning_rate=0.01,
                          n_neurons=90, random_state=42,
                          batch_norm_momentum=0.95)
dnn_clf_bn.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_valid1, y_valid=y_valid1)

0	Validation loss: 0.046670	Best loss: 0.046670	Accuracy: 98.55%
1	Validation loss: 0.041354	Best loss: 0.041354	Accuracy: 98.67%
2	Validation loss: 0.038889	Best loss: 0.038889	Accuracy: 98.59%
3	Validation loss: 0.034669	Best loss: 0.034669	Accuracy: 98.91%
4	Validation loss: 0.053341	Best loss: 0.034669	Accuracy: 98.63%
5	Validation loss: 0.038009	Best loss: 0.034669	Accuracy: 99.02%
6	Validation loss: 0.031121	Best loss: 0.031121	Accuracy: 99.14%
7	Validation loss: 0.037958	Best loss: 0.031121	Accuracy: 98.91%
8	Validation loss: 0.032794	Best loss: 0.031121	Accuracy: 99.14%
9	Validation loss: 0.040406	Best loss: 0.031121	Accuracy: 98.79%
10	Validation loss: 0.046076	Best loss: 0.031121	Accuracy: 98.83%
11	Validation loss: 0.040224	Best loss: 0.031121	Accuracy: 99.02%
12	Validation loss: 0.035200	Best loss: 0.031121	Accuracy: 99.30%
13	Validation loss: 0.031570	Best loss: 0.031121	Accuracy: 99.10%
14	Validation loss: 0.036835	Best loss: 0.031121	Accuracy: 99.02%
15	Validation loss: 

DNNClassifier(activation=<function leaky_relu.<locals>.parameterized_leaky_relu at 0x7faa3b3d3048>,
       batch_norm_momentum=0.95, batch_size=500, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x7faaf003ec50>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=90,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [27]:
y_pred = dnn_clf_bn.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9898812998637867

Now see if we can find a good set of hyperparameters that will work even better with batchnorm:

In [28]:
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_neurons":[10, 30, 50, 70, 90, 100, 120, 140, 160],
    'batch_size':[10, 50, 100, 500],
    'learning_rate':[0.01, 0.02, 0.05, 0.1],
    'activation': [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    'batch_norm_momentum':[0.9, 0.95, 0.98, 0.99, 0.999],
}
rnd_search_bn = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50, cv=3,
                                  random_state=42, verbose=2)
rnd_search_bn.fit(X_train1, y_train1, X_valid=X_valid1, y_valid=y_valid1, n_epochs=1000)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] n_neurons=70, learning_rate=0.01, batch_size=50, batch_norm_momentum=0.99, activation=<function relu at 0x7faad4328730> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0	Validation loss: 0.093151	Best loss: 0.093151	Accuracy: 97.65%
1	Validation loss: 0.068133	Best loss: 0.068133	Accuracy: 97.89%
2	Validation loss: 0.076174	Best loss: 0.068133	Accuracy: 97.89%
3	Validation loss: 0.046437	Best loss: 0.046437	Accuracy: 98.51%
4	Validation loss: 0.046017	Best loss: 0.046017	Accuracy: 98.94%
5	Validation loss: 0.053967	Best loss: 0.046017	Accuracy: 98.44%
6	Validation loss: 0.047223	Best loss: 0.046017	Accuracy: 98.24%
7	Validation loss: 0.050170	Best loss: 0.046017	Accuracy: 98.63%
8	Validation loss: 0.051913	Best loss: 0.046017	Accuracy: 98.71%
9	Validation loss: 0.044914	Best loss: 0.044914	Accuracy: 98.71%
10	Validation loss: 0.047774	Best loss: 0.044914	Accuracy: 98.71%
11	Validation loss: 0.054404	Best loss: 0.044914	Accuracy: 98.59%
12	Validation loss: 0.060613	Best loss: 0.044914	Accuracy: 98.59%
13	Validation loss: 0.041087	Best loss: 0.041087	Accuracy: 98.83%
14	Validation loss: 0.053867	Best loss: 0.041087	Accuracy: 98.67%
15	Validation loss: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.3min remaining:    0.0s


0	Validation loss: 0.076625	Best loss: 0.076625	Accuracy: 97.54%
1	Validation loss: 0.061069	Best loss: 0.061069	Accuracy: 98.48%
2	Validation loss: 0.056638	Best loss: 0.056638	Accuracy: 98.55%
3	Validation loss: 0.046330	Best loss: 0.046330	Accuracy: 98.63%
4	Validation loss: 0.075376	Best loss: 0.046330	Accuracy: 98.16%
5	Validation loss: 0.058886	Best loss: 0.046330	Accuracy: 98.36%
6	Validation loss: 0.058584	Best loss: 0.046330	Accuracy: 98.75%
7	Validation loss: 0.050245	Best loss: 0.046330	Accuracy: 98.63%
8	Validation loss: 0.042241	Best loss: 0.042241	Accuracy: 98.98%
9	Validation loss: 0.067637	Best loss: 0.042241	Accuracy: 98.36%
10	Validation loss: 0.061154	Best loss: 0.042241	Accuracy: 98.63%
11	Validation loss: 0.056852	Best loss: 0.042241	Accuracy: 98.71%
12	Validation loss: 0.038329	Best loss: 0.038329	Accuracy: 98.75%
13	Validation loss: 0.040109	Best loss: 0.038329	Accuracy: 98.83%
14	Validation loss: 0.038386	Best loss: 0.038329	Accuracy: 99.02%
15	Validation loss: 

...

13	Validation loss: 0.034259	Best loss: 0.028647	Accuracy: 99.06%
14	Validation loss: 0.034237	Best loss: 0.028647	Accuracy: 99.18%
15	Validation loss: 0.033698	Best loss: 0.028647	Accuracy: 99.06%
16	Validation loss: 0.036913	Best loss: 0.028647	Accuracy: 99.14%
17	Validation loss: 0.043418	Best loss: 0.028647	Accuracy: 98.94%
18	Validation loss: 0.029691	Best loss: 0.028647	Accuracy: 99.30%
19	Validation loss: 0.051480	Best loss: 0.028647	Accuracy: 98.71%
20	Validation loss: 0.043626	Best loss: 0.028647	Accuracy: 99.22%
21	Validation loss: 0.042409	Best loss: 0.028647	Accuracy: 99.14%
22	Validation loss: 0.039135	Best loss: 0.028647	Accuracy: 99.18%
23	Validation loss: 0.038548	Best loss: 0.028647	Accuracy: 99.22%
24	Validation loss: 0.042177	Best loss: 0.028647	Accuracy: 99.02%
25	Validation loss: 0.034609	Best loss: 0.028647	Accuracy: 99.06%
26	Validation loss: 0.045147	Best loss: 0.028647	Accuracy: 98.75%
Early stopping!
[CV]  n_neurons=100, learning_rate=0.01, batch_size=100, bat

3	Validation loss: 0.080104	Best loss: 0.080104	Accuracy: 98.40%
4	Validation loss: 0.103104	Best loss: 0.080104	Accuracy: 97.89%
5	Validation loss: 0.071464	Best loss: 0.071464	Accuracy: 98.32%
6	Validation loss: 0.067513	Best loss: 0.067513	Accuracy: 98.44%
7	Validation loss: 0.074298	Best loss: 0.067513	Accuracy: 98.59%
8	Validation loss: 0.083893	Best loss: 0.067513	Accuracy: 98.05%
9	Validation loss: 0.063978	Best loss: 0.063978	Accuracy: 98.51%
10	Validation loss: 0.073783	Best loss: 0.063978	Accuracy: 98.67%
11	Validation loss: 0.048352	Best loss: 0.048352	Accuracy: 98.75%
12	Validation loss: 0.094284	Best loss: 0.048352	Accuracy: 97.89%
13	Validation loss: 0.049250	Best loss: 0.048352	Accuracy: 98.75%
14	Validation loss: 0.059719	Best loss: 0.048352	Accuracy: 98.51%
15	Validation loss: 0.068560	Best loss: 0.048352	Accuracy: 98.51%
16	Validation loss: 0.078193	Best loss: 0.048352	Accuracy: 98.55%
17	Validation loss: 0.058747	Best loss: 0.048352	Accuracy: 98.87%
18	Validation los

51	Validation loss: 0.045921	Best loss: 0.031837	Accuracy: 98.63%
52	Validation loss: 0.047760	Best loss: 0.031837	Accuracy: 98.75%
53	Validation loss: 0.038447	Best loss: 0.031837	Accuracy: 98.79%
54	Validation loss: 0.063286	Best loss: 0.031837	Accuracy: 98.40%
55	Validation loss: 0.043216	Best loss: 0.031837	Accuracy: 98.67%
56	Validation loss: 0.046677	Best loss: 0.031837	Accuracy: 98.71%
Early stopping!
[CV]  n_neurons=50, learning_rate=0.01, batch_size=10, batch_norm_momentum=0.98, activation=<function elu at 0x7faad437d400>, total= 9.3min
[CV] n_neurons=50, learning_rate=0.01, batch_size=10, batch_norm_momentum=0.98, activation=<function elu at 0x7faad437d400> 
0	Validation loss: 0.100006	Best loss: 0.100006	Accuracy: 97.19%
1	Validation loss: 0.070636	Best loss: 0.070636	Accuracy: 98.01%
2	Validation loss: 0.064861	Best loss: 0.064861	Accuracy: 98.01%
3	Validation loss: 0.058831	Best loss: 0.058831	Accuracy: 98.28%
4	Validation loss: 0.047090	Best loss: 0.047090	Accuracy: 98.36

36	Validation loss: 0.115415	Best loss: 0.041111	Accuracy: 98.05%
37	Validation loss: 0.072906	Best loss: 0.041111	Accuracy: 98.55%
38	Validation loss: 0.048402	Best loss: 0.041111	Accuracy: 98.83%
39	Validation loss: 0.064833	Best loss: 0.041111	Accuracy: 98.87%
40	Validation loss: 0.063998	Best loss: 0.041111	Accuracy: 98.51%
41	Validation loss: 0.058144	Best loss: 0.041111	Accuracy: 98.75%
42	Validation loss: 0.044643	Best loss: 0.041111	Accuracy: 98.98%
43	Validation loss: 0.115881	Best loss: 0.041111	Accuracy: 97.62%
44	Validation loss: 0.053908	Best loss: 0.041111	Accuracy: 98.79%
45	Validation loss: 0.064171	Best loss: 0.041111	Accuracy: 98.71%
46	Validation loss: 0.074080	Best loss: 0.041111	Accuracy: 98.55%
47	Validation loss: 0.049487	Best loss: 0.041111	Accuracy: 98.94%
48	Validation loss: 0.059349	Best loss: 0.041111	Accuracy: 98.71%
49	Validation loss: 0.065244	Best loss: 0.041111	Accuracy: 98.79%
50	Validation loss: 0.055533	Best loss: 0.041111	Accuracy: 98.94%
51	Validat

KeyboardInterrupt: 